In [27]:
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [28]:
# Load data
X_train = np.load("Data/X_train.npy")
Y_train = np.load("Data/Y_train.npy")

print(X_train.shape)
print(Y_train.shape)

(700, 6)
(700,)


In [29]:
# Test polynomial degrees from 1 to 10
for degree in range(1, 11):
    # Generate polynomial features
    poly = PolynomialFeatures(degree=degree, include_bias=False)
    X_poly = poly.fit_transform(X_train)
    
    print(f"Degree {degree}: X_poly shape = {X_poly.shape}")
    
    # Fit regression model
    model = LinearRegression()
    model.fit(X_poly, Y_train)
    
    # Predict on training data
    Y_pred = model.predict(X_poly)
    
    # Compute R^2
    r2 = r2_score(Y_train, Y_pred)
    
    print(f"Degree {degree}: R^2 = {r2:.4f}\n")

Degree 1: X_poly shape = (700, 6)
Degree 1: R^2 = 0.5600

Degree 2: X_poly shape = (700, 27)
Degree 2: R^2 = 0.8298

Degree 3: X_poly shape = (700, 83)
Degree 3: R^2 = 0.8832

Degree 4: X_poly shape = (700, 209)
Degree 4: R^2 = 0.9817

Degree 5: X_poly shape = (700, 461)
Degree 5: R^2 = 0.9959

Degree 6: X_poly shape = (700, 923)
Degree 6: R^2 = 0.9998

Degree 7: X_poly shape = (700, 1715)
Degree 7: R^2 = 1.0000

Degree 8: X_poly shape = (700, 3002)
Degree 8: R^2 = 0.9999

Degree 9: X_poly shape = (700, 5004)
Degree 9: R^2 = 0.9994

Degree 10: X_poly shape = (700, 8007)
Degree 10: R^2 = 0.9983



Split the data into train set (600 entries) and test set (100 entries)

In [ ]:
# Load data
X = np.load("Data/X_train.npy")  # shape (700, 6)
Y = np.load("Data/Y_train.npy")  # shape (700,)
    
# Split data: first 500 train, last 200 test
X_train, X_test = X[:600], X[600:]
Y_train, Y_test = Y[:600], Y[600:]

print("Train shape:", X_train.shape, Y_train.shape)
print("Test shape:", X_test.shape, Y_test.shape)


Train shape: (600, 6) (600,)
Test shape: (100, 6) (100,)


In [31]:
# Loop over polynomial degrees
for degree in range(1, 11):
    poly = PolynomialFeatures(degree=degree, include_bias=False)
    X_train_poly = poly.fit_transform(X_train)
    X_test_poly = poly.transform(X_test)

    model = LinearRegression()
    model.fit(X_train_poly, Y_train)

    # Predictions
    Y_pred_train = model.predict(X_train_poly)
    Y_pred_test = model.predict(X_test_poly)

    # R² scores
    r2_train = r2_score(Y_train, Y_pred_train)
    r2_test = r2_score(Y_test, Y_pred_test)

    print(f"Degree {degree}: R² train = {r2_train:.4f}, R² test = {r2_test:.4f}")

Degree 1: R² train = 0.5591, R² test = 0.5613
Degree 2: R² train = 0.8302, R² test = 0.8151
Degree 3: R² train = 0.8891, R² test = 0.8058
Degree 4: R² train = 0.9834, R² test = 0.9223
Degree 5: R² train = 0.9977, R² test = 0.2496
Degree 6: R² train = 1.0000, R² test = -3.1742
Degree 7: R² train = 1.0000, R² test = -626.6045
Degree 8: R² train = 1.0000, R² test = -545568.5236
Degree 9: R² train = 0.9997, R² test = -12926719.5196
Degree 10: R² train = 0.9991, R² test = -36990070.4694
